In [84]:
with open("exampleinput19.txt", "r") as f:
    lines = f.readlines()
wkflows, parts = [l.removesuffix("\n") for l in lines[:lines.index("\n")]], [l.removesuffix("\n") for l in lines[lines.index("\n")+1:]]
keys, wkflows = [w[:w.find("{")] for w in wkflows], [w[w.find("{"):].strip("{}").split(',') for w in wkflows]

for i in range(len(wkflows)):
    wkflow_str = ""
    for instr in wkflows[i]:
        if ":"in instr:
            cond, outcome = instr[:instr.find(":")], instr[instr.find(":")+1:]
            wkflow_str += "\'" + outcome + "\' if " + cond + " else "
        else:
            wkflow_str += "\'"+instr+"\'"
    wkflows[i] = wkflow_str
wkflow_dict = {k:wf for k,wf in zip(keys,wkflows)}
del keys, wkflows

the_sum = 0
for part in parts:
    x, m, a, s = [int(item[2:]) for item in part.strip("{}").split(",")]
    wflow = eval(wkflow_dict["in"])
    while wflow not in "RA":
        wflow = eval(wkflow_dict[wflow])
    if wflow=="A":
        the_sum += x+m+a+s
the_sum

19114

In [153]:
from copy import deepcopy

def dict_maker(lines):#makes dict with lists as values
    wkflows = [l.removesuffix("\n") for l in lines[:lines.index("\n")]]
    keys, wkflows = [w[:w.find("{")] for w in wkflows], [w[w.find("{"):].strip("{}").split(',') for w in wkflows]
    for i in range(len(wkflows)):
        for j in range(len(wkflows[i])):
            if ":" in wkflows[i][j]:
                wkflows[i][j] = [wkflows[i][j][0], wkflows[i][j][1], int(wkflows[i][j][2:wkflows[i][j].find(":")]), wkflows[i][j][wkflows[i][j].find(":")+1:]]
            else:
                wkflows[i][j] = [wkflows[i][j]]    
    wkflow_dict = {k:wf for k,wf in zip(keys,wkflows)}
    return wkflow_dict

def get_accepted_ranges(wkflow_dict, low=1, high=4000):#dict with lists for values
    xmas_ind = {'x':0, 'm':1, 'a':2, 's':3}
    paths = [[[low,high+1] for _ in range(4)]+["in"]] #conditions on x,m,a,s to be on workflow of last item, all begin at "in" with no conditions
    accepted_ranges = []

    while paths!=[]:
        previous = deepcopy(paths[0])
        if previous[-1] in "RA":
            if previous[-1]=="A":
                accepted_ranges.append(previous[:-1])
            paths.remove(previous)
            continue
        current = deepcopy(previous)
        for instr in wkflow_dict[current[-1]]:
            if len(instr)==4:
                following = deepcopy(current)
                categ, comp, threshold, outcome = instr
                if comp=="<":
                    if threshold < current[xmas_ind[categ]][1]:
                        current[xmas_ind[categ]][1] = threshold #pythonic intervals
                    if threshold > following[xmas_ind[categ]][0]: 
                        following[xmas_ind[categ]][0] = threshold #pythonic intervals
                elif comp==">":
                    if threshold > current[xmas_ind[categ]][0]:
                        current[xmas_ind[categ]][0] = threshold+1 #pythonic intervals
                    if threshold < following[xmas_ind[categ]][1]:
                        following[xmas_ind[categ]][1] = threshold+1 #pythonic intervals
                else:
                    raise ValueError("Comparator not < nor >")
                current[-1] = outcome
                paths.append(current)
                current = deepcopy(following)
            elif len(instr)==1:
                current[-1] = instr[0]
                paths.append(current)
            else:
                raise IndexError("Workflow instruction has neither 4 elements (standard) nor 1 (end-of-line)")
        paths.remove(previous)
    return accepted_ranges

with open("input19.txt", "r") as f:
    lines = f.readlines()

wkflow_dict = dict_maker(lines)
accepted_ranges = get_accepted_ranges(wkflow_dict)
the_sum = 0 
for list in accepted_ranges:
    increment = 1
    for rng in list:
        increment *=(rng[1]-rng[0])
    the_sum +=increment #the ranges have no overlap, just add their numbers
the_sum

127517902575337

In [152]:
from copy import deepcopy

def dict_maker(lines):#makes dict with strings as values
    wkflows = [l.removesuffix("\n") for l in lines[:lines.index("\n")]]
    keys, wkflows = [w[:w.find("{")] for w in wkflows], [w[w.find("{"):].strip("{}").split(',') for w in wkflows]
    wkflow_dict = {k:wf for k,wf in zip(keys,wkflows)}
    return wkflow_dict

def get_accepted_rngs(wkflow_dict, low=1, high=4000):#dict with strings for values
    xmas_ind = {'x':0, 'm':1, 'a':2, 's':3}
    paths = [[[low,high+1] for _ in range(4)]+["in"]] #conditions on x,m,a,s to be on workflow of last item, all begin at "in" with no conditions
    accepted_ranges = []

    while paths!=[]:
        previous = deepcopy(paths[0])
        if previous[-1] in "RA":
            if previous[-1]=="A":
                accepted_ranges.append(previous[:-1])
            paths.remove(previous)
            continue
        current = deepcopy(previous)
        for instr in wkflow_dict[current[-1]]:
            if ":" in instr:
                following = deepcopy(current)
                categ, comp, threshold, outcome = instr[0], instr[1], int(instr[2:instr.find(":")]), instr[instr.find(":")+1:]
                if comp=="<":
                    if threshold < current[xmas_ind[categ]][1]:
                        current[xmas_ind[categ]][1] = threshold #pythonic intervals
                    if threshold > following[xmas_ind[categ]][0]: 
                        following[xmas_ind[categ]][0] = threshold #pythonic intervals
                elif comp==">":
                    if threshold > current[xmas_ind[categ]][0]:
                        current[xmas_ind[categ]][0] = threshold+1 #pythonic intervals
                    if threshold < following[xmas_ind[categ]][1]:
                        following[xmas_ind[categ]][1] = threshold+1 #pythonic intervals
                else:
                    raise ValueError("Comparator not < nor >")
                current[-1] = outcome
                paths.append(current)
                current = deepcopy(following)
            else:
                current[-1] = instr
                paths.append(current)
        paths.remove(previous)
    return accepted_ranges

with open("input19.txt", "r") as f:
    lines = f.readlines()
wkflow_dict = dict_maker(lines)
accepted_ranges = get_accepted_rngs(wkflow_dict)
the_sum = 0 
for list in accepted_ranges:
    increment = 1
    for rng in list:
        increment *=(rng[1]-rng[0])
    the_sum +=increment #the ranges have no overlap, just add their numbers
the_sum

127517902575337